In [ ]:
#Cell_1.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

import warnings
warnings.simplefilter('ignore')

import plotly

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode()

from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error


plt.rcParams['font.family'] = 'Noto Sans' # for Japanese in graph (L)

In [ ]:
#Cell_2.
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
from greykite.algo.forecast.silverkite.constants.silverkite_column import SilverkiteColumn
from greykite.common.features.timeseries_features import get_available_holidays_across_countries
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results

In [ ]:
#Cell_3.
mufj_csv="8316_2018.csv"
mufj=pd.read_csv(mufj_csv,skiprows=1,header=0,delimiter=',',encoding="shift-jis")
print(mufj.shape) #確認
print(mufj.head()) #確認
print(type(mufj['日付'][0]))
mufj['日付']=pd.to_datetime(mufj['日付'])
print(type(mufj['日付'][0]))

In [ ]:
#Cell_4.
ts = UnivariateTimeSeries()
ts.load_data(
    df=mufj,
    time_col="日付", # 日付の列
    value_col="終値", # 値の列
    freq="D" # 日付の単位
)

In [ ]:
#Cell_5.
fig = ts.plot()
plotly.io.show(fig)

In [ ]:
#Cell_6 課題2 Cell6で一週間ごとの移動平均をプロットしている。 これを一か月ごとに変更して実行せよ。
fig = ts.plot_grouping_evaluation(
    # デフォルトでは平均を計算する(任意の集計方法を指定可能)
    groupby_sliding_window_size=30, # 集計する粒度(7の場合、t-3~t+3の7日間の平均を計算)
    title="Monthly average")
plotly.io.show(fig)

In [ ]:
#Cell_6.
# fig = ts.plot_grouping_evaluation(
#     # デフォルトでは平均を計算する(任意の集計方法を指定可能)
#     groupby_sliding_window_size=7, # 集計する粒度(7の場合、t-3~t+3の7日間の平均を計算)
#     title="Weekly average")
# plotly.io.show(fig)

In [ ]:
#Cell_7.
fig = ts.plot_quantiles_and_overlays(
    groupby_time_feature="dow", # "day of week"の略
    show_mean=True, # 平均も描画
    show_quantiles=[0.1, 0.5, 0.9], # 各分位点も描画 # type: ignore
    center_values=True # 中央化の有無
)
plotly.io.show(fig)

In [ ]:
#Cell_7.
fig = ts.plot_quantiles_and_overlays(
    groupby_time_feature="woy", # "week of year"の略
    show_mean=True, # 平均も描画
    show_quantiles=[0.1, 0.5, 0.9], # 各分位点も描画 # type: ignore
    center_values=True # 中央化の有無
)
plotly.io.show(fig)

In [ ]:
#Cell_7.
fig = ts.plot_quantiles_and_overlays(
    groupby_time_feature="doy", # "day of year"の略
    show_mean=True, # 平均も描画
    show_quantiles=[0.1, 0.5, 0.9], # 各分位点も描画 # type: ignore
    center_values=True # 中央化の有無
)
plotly.io.show(fig)

In [ ]:
#Cell_8.
metadata = MetadataParam(
    time_col="日付", # 日付の列名
    value_col="終値", # 値の列名
    freq="D" # 日付の単位
)

In [ ]:
#Cell_9.
forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=mufj,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=30,
        coverage=0.95, # 予測区間を95%に設定
        metadata_param=metadata
    )
)

In [ ]:
result.backtest.test_evaluation

In [ ]:
#Cell_10.
print(result.model[-1].summary(max_colwidth=50))

In [ ]:
#Cell_11.
backtest = result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
#Cell_12.
fig = result.forecast.plot_components()
plotly.io.show(fig)

In [ ]:
#Cell_13.
model = ChangepointDetector()
res = model.find_trend_changepoints(
    df=mufj, # データフレーム
    time_col="日付", # 日付の列名
    value_col="終値", # 値の列名
    yearly_seasonality_order=0, # 年周期を表現するためのフーリエ級数の次数(今回は年周期を仮定してないので0)
    regularization_strength=0.5, # adaptive lassoの罰則の強さ(0~1の値)
    resample_freq="7D", # データを括る粒度
    potential_changepoint_n=20, # 変化点の数、値を大きくすると計算負荷が高くなる
    yearly_seasonality_change_freq=None, # 年周期の値が変わる間隔(今回は年周期ないので無視)
    no_changepoint_distance_from_end="14D", # ラスト14日は変化点なし
    actual_changepoint_min_distance="7D" # 変化点どうしの感覚は最低7日空ける
)

fig = model.plot(
    observation=True,
    trend_estimate=False,
    trend_change=True,
    yearly_seasonality_estimate=False,
    adaptive_lasso_estimate=True,
    plot=False)
plotly.io.show(fig)

In [ ]:
#Cell_14.
growth = {
    "growth_term": "linear" # 線形トレンド
}

changepoints = {
    "changepoints_dict": {
        "method": "auto",
        "yearly_seasonality_order": 0,
        "regularization_strength": 0.5,
        "resample_freq": "7D",
        "potential_changepoint_n": 20,
        "yearly_seasonality_change_freq": None,
        "no_changepoint_distance_from_end": "14D",
        "actual_changepoint_min_distance": "7D"
    }
}

seasonality = {
    "yearly_seasonality": False,
    "quarterly_seasonality": False,
    "monthly_seasonality": False,
    "weekly_seasonality": 2, # 週周期の次数
    "daily_seasonality": False
}

events = {
    "holiday_lookup_countries": None # 祝日なし
}

model_components = ModelComponentsParam(
    seasonality=seasonality,
    growth=growth,
    changepoints=changepoints,
    autoregression={
        "autoreg_dict": None # 自己回帰の項なし
    },
    events=events,
    regressors=None, # 説明変数なし
    lagged_regressors={
        "lagged_regressor_dict": None # 説明変数のラグも当然なし
    },
    custom={
        "fit_algorithm_dict": {
            "fit_algorithm": "ridge"
        },
        "feature_sets_enabled": {
            SilverkiteColumn.COLS_DAY_OF_WEEK: False, # 曜日ダミーの有無
            SilverkiteColumn.COLS_TREND_WEEKEND: False, # 平日/週末とトレンドとの交互作用の有無
            SilverkiteColumn.COLS_TREND_DAY_OF_WEEK: False, # 曜日とトレンドとの交互作用の有無
            SilverkiteColumn.COLS_TREND_WEEKLY_SEAS: False # 週周期の変化を許容するかどうか
        }
    },
    uncertainty={
        "uncertainty_dict": {
            "uncertainty_method": "simple_conditional_residuals",
            "params": {
                "quantiles": [0.025, 0.975],
                "conditional_cols": ["year"], # 予測区間を月ごとに計算する
                "quantile_estimation_method": "normal_fit"
            }

        },
    }
)


# 実行
custom_result = forecaster.run_forecast_config(
    df=mufj,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=14,
        coverage=0.95,  # 95% prediction intervals
        model_components_param=model_components,
        metadata_param=metadata
    )
)

In [ ]:
#Cell_15.
backtest = custom_result.backtest
fig = backtest.plot()
plotly.io.show(fig)

In [ ]:
#Cell_16.
print(custom_result.model[-1].summary(max_colwidth=50))

In [ ]:
#Cell_17.
fig = custom_result.forecast.plot_components()
plotly.io.show(fig)

In [ ]:
#Cell_18.
custom_result.backtest.test_evaluation